### Stitch Together Bank and Beals data from Rent Statements

Creates continuous data of bank and Beals data from Rent Statements.

In [1]:
taxreturn_yr='TaxReturn_2022-2023'

In [2]:
import numpy as np
import pandas as pd

In [3]:
spreadsheets=[ \
               'Beals_RentalStatement_APR2022_21062022_2173', 'Beals_RentalStatement_MAY2022_21062022_2174', \
               'Beals_RentalStatement_JUN2022_11092022_2183', 'Beals_RentalStatement_JUL2022_11092022_2184' \
             ]


In [4]:
#path = 'M:/My Documents/Business/Property/Statements/processed/'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\processed\\'

In [5]:
def strip(df):
    df['Memo']=df['Memo'].str.strip()
    df['Subcategory']=df['Subcategory'].str.strip()
    if 'Description' in df.columns:
        df['Description']=df['Description'].str.strip()
    return df

def load_bank_worksheet(spreadsheet, worksheet):
    input_file = path + spreadsheet + '.xlsm'
    df = pd.read_excel(input_file,sheet_name=worksheet)
    df.drop('Import', axis=1, inplace=True)
    df.drop('Number', axis=1, inplace=True)
    df = df.astype({"Description": str})
    df['Amount']= round(df['Amount'],2)
    df=df.replace('nan', '')
    df = df[~df.Account.isnull()]
    return df

def load_beals_worksheet(spreadsheet, worksheet):
    input_file = path + spreadsheet + '.xlsm'
    df = pd.read_excel(input_file,sheet_name=worksheet)
    #df.drop('Import', axis=1, inplace=True)
    #df.drop('Number', axis=1, inplace=True)
    #df = df.astype({"Description": str})
    df = df[~df.ID.isnull()]
    return df

In [6]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [7]:
df6 = pd.DataFrame(columns=['Date','Account','Amount','Subcategory','Memo','Property','Description'])
df3 = pd.DataFrame(columns=['Date','Account','Amount','Subcategory','Memo','Property','Description'])
dfB = pd.DataFrame(columns=['ID','Activity I D','Date Created','Item Description','Document Type','Document Number','Debit Amount','Credit Amount','ID Check'])
for spreadsheet in spreadsheets:
    df6Temp=load_bank_worksheet(spreadsheet, 'MT')
    df6=pd.concat([df6,df6Temp])
    df3Temp=load_bank_worksheet(spreadsheet, 'IV')
    df3=pd.concat([df3,df3Temp])
    dfBTemp=load_beals_worksheet(spreadsheet, 'Beals')
    dfB=pd.concat([dfB,dfBTemp])

#df6=strip(df6)
#df3=strip(df3)
#dfB=strip(df40)

df6=df6.loc[:, ~df6.columns.str.contains('^Unnamed')]
df3=df3.loc[:, ~df3.columns.str.contains('^Unnamed')]
dfB=dfB.loc[:, ~dfB.columns.str.contains('^Unnamed')]
    
    
dfB=dfB[['ID','Activity I D','Date Created','Item Description','Document Type','Document Number','Debit Amount','Credit Amount','ID Check']]
df6=df6[['Date','Account','Amount','Subcategory','Memo','Property','Description']]
df3=df3[['Date','Account','Amount','Subcategory','Memo','Property','Description']]

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [8]:
#llp_path = 'M:/My Documents/Business/Bugisiw Ltd/TaxReturn/'+taxreturn_yr+'/LLP accounts/'
llp_path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'
file6=backup_file(llp_path+'6045.xlsx')
file3=backup_file(llp_path+'3072.xlsx')
fileB=backup_file(llp_path+'beals.xlsx')
#df6.to_csv(file6, index=False)
#df3.to_csv(file3, index=False)
#dfB.to_csv(fileB, index=False)
df6.to_excel(file6,sheet_name='6045', index=False)
df3.to_excel(file3,sheet_name='3072', index=False)
dfB.to_excel(fileB,sheet_name='beals', index=False)

llp_path = 'M:\My Documents\Business\Bugisiw Ltd\TaxReturn\TaxReturn_2022-2023\LLP accounts'
bank_output_file = llp_path + '/transactions_gen.xlsx'
writer = pd.ExcelWriter(bank_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
df6.to_excel(writer,sheet_name='6045', index=False)
df3.to_excel(writer,sheet_name='3072', index=False)
dfB.to_excel(writer,sheet_name='Beals', index=False)
writer.save()